# Example GATIS Conversion Pipeline with Existing Active Transportation Data from Austin, Texas 
---


This notebook demonstrates a potential workflow for converting existing active transportation data into GATIS format. Data are converted into GATIS format using our best guess at what the mapping would be, so there may be certain attributes that were misinterpreted or missed.

The final sample data can be accessed using the following links:
<ul>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=936f374afa8d4612ba1fafaf3eedc000'>Sample GATIS Edges (Austin, TX) Feature Layer</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=bd0e32d7b09e4cd6a9cb41a6ac8b8864'>Sample GATIS Edges (Austin, TX) GeoJSON Download</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=1c563a62a83244ba9edbeaa6eafee6a2'>Sample GATIS Nodes (Austin, TX) Feature Layer</a></li>
    <li><a href='https://usdot.maps.arcgis.com/home/item.html?id=5b63f3dd5adf4503a756c2c0e9738a2c'>Sample GATIS Nodes (Austin, TX) GeoJSON Download</a></li>
</ul>

This notebook can be run using Jupyter Notebooks with a Python environment with the latest version of GeoPandas installed. The outputs of this notebook are retained in this file for reference.

Feel free to create a copy of this notebook and modify as needed to convert other data sources to GATIS.

## Data Sources
The table below provides links to the various datasets used to create the sample GATIS data. To re-run this notebook these data needs to be downloaded and placed in a folder called `data` within the same directory as this notebook.

| Filename | Description |
| -- | -- |
| [Sidewalks_20250825.geojson](https://data.austintexas.gov/dataset/Sidewalks/vchz-d9ng/about_data) | Latest and most complete version of Austin sidewalks |
| [TRANSPORTATION.markings_short_line_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION-markings_short_line/9hak-anfp/about_data) | Only kept crosswalks |
| [TRANSPORTATION_curb_ramps.geojson](https://hub.arcgis.com/datasets/a477eed132a04d26b70e74d8fba74c42_0/explore?location=30.322658%2C-97.746550%2C10.83&uiVersion=content-views) | Has curb ramps |
| [TRANSPORTATION_asmp_street_network_20250825.geojson](https://data.austintexas.gov/Locations-and-Maps/TRANSPORTATION_asmp_street_network/cr24-n8br/about_data) | This one appears to conflict partially with the bicycle facilities one but contains more streets |
| [TRANSPORTATION_urban_trails_network_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_urban_trails_network/jdwm-wfps/about_data) | Centerlines for bikeped trails that aren't in the other files |

<!-- These were not included:
| [Completed_Sidewalks_2021_09_30_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/Completed_Sidewalks_2021_09_30/6pkw-7frd/about_data) | Used Sidewalks_20250825.geojson instead |
| [Flashing Beacons_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/Flashing-Beacons/wczq-5cer/about_data) | Contains PBHs in point format, would need to be spatially joined to crosswalk edges to be compliant with GATIS |
| [Roadway Sign Assets_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/Roadway-Sign-Assets/ggnk-3ykn/about_data) | Signs haven't made it into GATIS yet |
| [Traffic Signals and Pedestrian Signals_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/Traffic-Signals-and-Pedestrian-Signals/p53x-x73x/about_data) | Needs to be spatially joined to the crosswalk edges |
| [TRANSPORTATION_bicycle_facilities_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_bicycle_facilities/23hw-a95n/about_data) | Overlaps with asmp_street_network and urban_trails |
| [TRANSPORTATION_markings_long_line_20250825.csv](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_markings_long_line/nxkm-x3yr/about_data) | Didn't have a geometry column, might need to be joined to one of the other datasets? Does have bike facilities. |
| [TRANSPORTATION_markings_specialty_line_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_markings_specialty_line/rczx-h2ey/about_data) | Contains some bike facility separation components that could be joined to bike facilities |
| [TRANSPORTATION_markings_specialty_point_20250825.geojson](https://data.austintexas.gov/Transportation-and-Mobility/TRANSPORTATION_markings_specialty_point/53jy-y8pj/about_data) | Points for thermoplastic/paint pictogram markings on the road. Not applicable to GATIS. | -->

## Import required packages

In [1]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import json
import shapely
from shapely.ops import Point, LineString

# optionally use bounding box as a mask
# bbox = shapely.from_wkt("POLYGON((-97.748244 30.278771, -97.736805 30.278771, -97.736805 30.267221, -97.748244 30.267221, -97.748244 30.278771))")
# bbox = shapely.from_wkt("POLYGON((-97.729976 30.258163, -97.718538 30.258163, -97.718538 30.246611, -97.729976 30.246611, -97.729976 30.258163))")
bbox = None

## Writes all downloaded GeoJSON Files to GPKG for viewing in QGIS

In [2]:
# re-write all geojson to gpkg for efficiency
austin_layers = list(Path.cwd().glob('data/*'))
for layer in austin_layers:
    if layer.suffix != '.geojson':
        continue
    if layer.name.split(layer.suffix)[0] in gpd.list_layers("data/austin_all_data.gpkg")['name'].tolist():
        print(f"{layer.name.split(layer.suffix)[0]} layer already exists")
        continue
    print("writing layer:", layer.name)
    gdf = gpd.read_file(layer)
    gdf.to_file("data/austin_all_data.gpkg", layer=layer.name.split('.geojson')[0], driver="GPKG")

Completed_Sidewalks_2021_09_30_20250825 layer already exists
Flashing Beacons_20250825 layer already exists
Roadway Sign Assets_20250825 layer already exists
Sidewalks_20250825 layer already exists
Traffic Signals and Pedestrian Signals_20250825 layer already exists
TRANSPORTATION.markings_short_line_20250825 layer already exists
TRANSPORTATION_asmp_street_network_20250825 layer already exists
TRANSPORTATION_bicycle_facilities_20250825 layer already exists
TRANSPORTATION_curb_ramps layer already exists
TRANSPORTATION_markings_specialty_line_20250825 layer already exists
TRANSPORTATION_markings_specialty_point_20250825 layer already exists
TRANSPORTATION_urban_trails_network_20250825 layer already exists


## Helper functions for retreiving coded values from an ArcGIS feature service

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

def get_attribute_metadata(api_endpoint):
    '''
    Gets all the field names from an ArcGIS REST feature layer
    '''
    
    params = {
        'f': 'json'
    }
    response = requests.get(api_endpoint,params=params,headers=headers)
    
    attributes = {}

    for x in response.json()['fields']:
        attributes[x.get('name')] = {
            'attribute_alias': x.get('alias'),
            'data_type': x.get('type'),
            'codedValues': parse_coded_values(x)
        }
    
    return attributes

def parse_coded_values(field_dict):
    '''
    Gets coded values for a field if not every possible value for a field is represented in the data
    '''    
    codedValues = access_key(field_dict,['domain','codedValues'])

    # if there is no coded values entry
    if isinstance(codedValues,list):
        if len(codedValues) > 0:
            codedValues = {x['code']:x['name'] for x in codedValues}
            return codedValues
    else:
        return None
    
def access_key(response_json,levels):
    # handles errors when accessing multiple levels of a large json
    response_json = response_json.copy()
    for level in levels:
        if isinstance(response_json,dict) == False:
            return None
        response_json = response_json.get(level)
    return response_json

## GATIS Template

The GATIS template can be used to create a DataFrame/Table of the required/recommended fields at any particular Tier level. If desired, you can also decide to include optional fields. For this demonstration, we'll bring in the Tier 4 template so that we have all fields.

In [4]:
# import tier 4 GATIS template so that all of the possible attributes are in there
gatis_template_url = "https://raw.githubusercontent.com/dotbts/BPA/main/draft_gatis_specification/templates/tier_4_and_optional"
gatis_edges_geojson = requests.get(gatis_template_url + "/edge.geojson").json()
gatis_nodes_geojson = requests.get(gatis_template_url + "/node.geojson").json()

# use these lists to append the converted gatis data to
all_gatis_edges = []
all_gatis_nodes = []

def geojson_to_geopandas(geojson,feature_type=None,subfeature_type=None):
    """
    Converts GATIS template geojson to a geopandas dataframe. Optionally, specify the
    feature type and subfeature type to only return one feature type
    """
    
    # convert to pandas dataframe
    records = []
        
    # raise exception if feature_type or subfeature_type are not defined
    if feature_type is not None:
        accepted_feature_types = ["edge","node","point","zone"]
        if feature_type not in accepted_feature_types:
            raise Exception(f"feature type is not one of {accepted_feature_types}")
    if subfeature_type is not None:
        accepted_subfeature_types = [x['properties'][f'{feature_type}_type'] for x in geojson['features']]
        if subfeature_type not in accepted_subfeature_types:
            raise Exception(f"subfeature type is not one of {accepted_subfeature_types}")

    for record in geojson['features']:
        record = record['properties']
        if (feature_type is not None) & (subfeature_type is not None):
            if record[f"{feature_type}_type"] is not None:
                if record[f"{feature_type}_type"] != subfeature_type:
                    continue
        record['geometry'] = None
        records.append(record)
    gdf = pd.DataFrame.from_records(records)

    # replace all NaN with None
    return gdf

def create_empty_gdf_like(empty_df_with_cols,gdf_to_mimic):
    """
    Helper function that copies the index and number of rows of the dataset you're trying to convert
    to GATIS
    """
    return gpd.GeoDataFrame([empty_df_with_cols.values[0] for x in range(0,gdf_to_mimic.shape[0])],geometry=gdf_to_mimic.geometry,columns=empty_df_with_cols.columns,index=gdf_to_mimic.index)

# Converting each layer to GATIS
The cells below show how each layer was converted to GATIS format. We went through the available attributes in each layer to see which ones could be mapped to a GATIS attribute value.

## Curb Ramps

| Name | Note |
| - | - |
| OBJECTID | dropped |
| CURB_RAMPS_ID | **added to reference_ids** |
| CREATED_BY | dropped |
| CREATED_DATE | dropped |
| MODIFIED_BY | dropped |
| MODIFIED_DATE | dropped |
| CURB_RAMP_TYPE | **added to ramp_type** |
| ADDRESS_DESCRIPTION | dropped |
| RATING | dropped |
| ASSESSMENT_DATE | **added to check_date** |
| CURB_RAMP_ADA_TYPE | **potential to add to curb_type/presence/ramp_type** |
| DETECTABLE_WARNING | **add to detectable_warning** | 
| POINT_X | dropped |
| POINT_Y | dropped |
| YEAR_BUILT | dropped, used data_construction_completed instead |
| PROJECT_STATUS | dropped |
| SUB_PROJECT_ID | dropped |
| FUND_DEPARTMENT_UNIT | dropped |
| SOURCE_OF_FUNDING | dropped |
| CONTRACT | dropped |
| WORK_ASSIGNMENT | dropped |
| ASSIGNED_FOR_CONSTRUCTION_DATE | dropped |
| CONSTRUCTION_START | dropped |
| DATE_CONSTRUCTION_COMPLETED | **added to date_built** |
| NOTES | dropped |
| MXADDRESSCODE | dropped |
| MXLOCATION | dropped |
| STATUS | **converted to status** |
| GLOBALID | dropped |
| ASSESSMENT_TYPE | dropped |
| LEVERAGING | dropped |
| CONSTRUCTION_MANAGER | dropped |
| geometry | **kept as is** |

In [5]:
curb_ramps = gpd.read_file("data/austin_all_data.gpkg", layer='TRANSPORTATION_curb_ramps', mask=bbox)

# get the coded values from the arcgis feature layer
curb_ramps_endpoint = "https://services.arcgis.com/0L95CJ0VTaxqcmED/arcgis/rest/services/TRANSPORTATION_curb_ramps/FeatureServer/0"
metadata = get_attribute_metadata(curb_ramps_endpoint)
coded_values = {key:item.get('codedValues') for key,item in metadata.items() if item.get('codedValues') is not None}
for key, codedValues in coded_values.items():
    curb_ramps[key] = curb_ramps[key].map(codedValues).fillna(curb_ramps[key])

keep_cols = [
    "CURB_RAMPS_ID", "CURB_RAMP_TYPE", "ASSESSMENT_DATE",
    "CURB_RAMP_ADA_TYPE", "DETECTABLE_WARNING", "DATE_CONSTRUCTION_COMPLETED",
    "STATUS", 'geometry'
]
curb_ramps = curb_ramps[keep_cols].copy()

# drop future ramps / never constructed
curb_ramps = curb_ramps[(curb_ramps["STATUS"].isin(['Never Constructed','Future']))==False].copy()

curb_ramps.reset_index(drop=True,inplace=True)

In [6]:
# create empty gatis geodataframes with the same structure
gatis_curb_ramps = geojson_to_geopandas(gatis_nodes_geojson,'node','curb_ramp')
gatis_curb_ramps = create_empty_gdf_like(gatis_curb_ramps,curb_ramps)

In [7]:
# reference_id helps capture the ID of the orginal data source
# can be a list of dicts with multiple IDs but be sure to add a "source" field that's helpful 
# for tracing the data back to the original source
# also optionally provide source_url if available
reference_id = curb_ramps[["CURB_RAMPS_ID"]].to_dict(orient='records')
gatis_curb_ramps['reference_ids'] = [[{'source': 'austin', 'source_url': '',**item}] for item in reference_id]

# grab the date built (use highest precision available up to day (don't include time))
gatis_curb_ramps['date_built'] = curb_ramps["DATE_CONSTRUCTION_COMPLETED"].apply(lambda x: pd.to_datetime(x).date().strftime(format="%Y-%m-%d") if pd.notnull(x) else None)

# get check_date
gatis_curb_ramps['check_date'] = curb_ramps['ASSESSMENT_DATE'].apply(lambda x: pd.to_datetime(x).date().strftime(format="%Y-%m-%d") if pd.notnull(x) else None)

# mark absent curb ramps
gatis_curb_ramps.loc[curb_ramps['CURB_RAMP_TYPE'] == 'ABSENT', 'presence'] = "no"

# ramp_type
gatis_curb_ramps.loc[curb_ramps['CURB_RAMP_TYPE']=='DIAGONAL','ramp_type'] = 'diagonal'
# NOTE: some of these might actually be parallel
gatis_curb_ramps.loc[curb_ramps['CURB_RAMP_TYPE']=='DIRECTIONAL','ramp_type'] = 'perpendicular'

# NOTE: GATIS doesn't currenlty have a comprehensive list of ada curb ramp types (but may add in draft 2.0)
# gatis_curb_ramps.loc[curb_ramps['ramp_type'].isnull() & curb_ramps['CURB_RAMP_ADA_TYPE'].notnull(), 'ramp_type'] = curb_ramps['CURB_RAMP_ADA_TYPE']

# NOTE making the assumption that these are true
def process_detectable_warning(x):
    if pd.isnull(x):
        return None
    # x = x.upper()
    if (x in ['NONE']) | (x is None):
        return "no"
    elif x in ['CONCRETE-SCORED']:
        return "tactile and not contrasted"
    elif x in ['BRICK','METAL_PLATE','BUTTONS']:
        return "tactile and contrasted"
    else:
        return None
gatis_curb_ramps['detectable_warning'] = curb_ramps["DETECTABLE_WARNING"].apply(lambda x: process_detectable_warning(x))

def process_status(x):
    if isinstance(x,str):
        if x in ["Active"]:
            return "open"
        elif x in ["Under Construction"]:
            return "under construction"
        elif x in ["Inactive-Decommissioned"]:
            return None
gatis_curb_ramps['status'] = curb_ramps["STATUS"].apply(lambda x: process_status(x))

In [8]:
# clean up and add to list
gatis_curb_ramps = gatis_curb_ramps.where(gatis_curb_ramps.notnull(), None)
all_gatis_nodes.append(gatis_curb_ramps)

## Streets

| Name | Note |
| - | - |
| creationdate | drop |
| mean_row | drop |
| name | **add to street name** |
| max_row | drop |
| improvement | drop |
| ped_popup | drop |
| sif_xs_general | drop |
| median_row | drop |
| street_level | **keep to drop interstates** |
| address_range | drop |
| objectid | drop |
| globalid | drop |
| creator | drop |
| bicycle_facility | **add to bikeway_type** |
| editor | drop |
| bicycle_popup | drop |
| in_table | drop |
| shape_length | drop |
| editdate | drop |
| rec_bicycle_facility | drop |
| ex_xs_general | drop |
| project_description | drop |
| exist_lanes | **add to thru_lanes** |
| asmp_street_network_id | **add to reference_ids** |
| min_row | drop |
| roadway_popup | drop but could contain some data that could apply to certain gatis fields |
| assum_lanes_fut | drop |
| remarks | drop |
| priority_network | drop |
| project_type_final | drop |
| segment_limits | drop |
| sort_order | drop |
| required_row_alpha | drop |
| council_district | drop |
| geometry | **kept as is** |

In [9]:
# explode used to convert multilinestrings to linestrings
streets = gpd.read_file("data/austin_all_data.gpkg", layer='TRANSPORTATION_asmp_street_network_20250825', mask=bbox).explode()

keep_cols = [
    'name', 'bicycle_facility', 'street_level',
    'exist_lanes', 'asmp_street_network_id', 'geometry'
]
streets = streets[keep_cols]

# drop trails because they appear in a different dataset too
streets = streets[streets['bicycle_facility'].isin(['Trail - Unpaved','Trail - Paved'])==False]

# drop if no bicycle facility
streets = streets[streets['bicycle_facility'].notna()]

# remove if roads bikes are not allowed?
# it looks like level=5 are the interstates/controlled access roads
streets = streets[streets['street_level']!='5']

streets.reset_index(drop=True,inplace=True)

In [10]:
# create empty gatis geodataframes with the same structure
gatis_roads = geojson_to_geopandas(gatis_edges_geojson,'edge','road')
gatis_roads = create_empty_gdf_like(gatis_roads,streets)

In [11]:
reference_id = streets[["asmp_street_network_id"]].to_dict(orient='records')
gatis_roads['reference_ids'] = [[{'source': 'austin',**item}] for item in reference_id]

# aligning these to NBN definitions
# NOTE need to check orientation of the bike lane with parking
convert_dict = {
    # 'Shared Lane', these are good as is
    # 'Bike Lane', 
    'Wide Shoulder': "Paved Shoulder",
    'Wide Curb Lane': "Shared Lane", 
    'Bike Lane - wParking': "Bike Lane", 
    'Shoulder': "Paved Shoulder",
    'Protected Bike Lane': "Separated Bike Lane", 
    'Bike Lane - Buffered': "Buffered Bike Lane", 
    'Sharrows': "Shared Lane",
    'Neighborhood Bikeway': "Shared Lane", 
    'Bike Lane-Protected 2': "Separated Bike Lane",
    'Bike Ramp to Sidewalk': "", 
    'Bike Lane - Climbing': "Bike Lane", 
    'Trail - Unpaved': None,
    'Trail - Paved': None, 
    'Bike Lane-Protected 1': "Separated Bike Lane", 
    'Bridge': None
}

gatis_roads.loc[:,"bikeway:left:bikeway_type"] = streets['bicycle_facility'].map(convert_dict)
gatis_roads.loc[:,"bikeway:right:bikeway_type"] = streets['bicycle_facility'].map(convert_dict)

# populuate other relevant fields
gatis_roads.loc[streets['bicycle_facility'] == 'Bridge','bridge'] = "yes"

# correct for facilities that are only on one side of the road
def process_bicycle_facility(row):
    update_dict = {}
    if row['bicycle_facility'] in ['Bike Lane-Protected 2']:
        #NOTE I'm just using the "left" as a demonstration, the actual data be "right"
        update_dict['bikeway:left:directionality'] = "both"
        # clear out the bicycle facility on the right
        update_dict['bikeway:right:bikeway_type'] = None
        update_dict['bikeway:right:presence'] = "no"
    elif row['bicycle_facility'] in ["Bike Lane - Climbing"]:
        update_dict['bikeway:right:bikeway_type'] = None
        update_dict['bikeway:right:presence'] = "no"
    return update_dict
re_map = streets.apply(lambda row: process_bicycle_facility(row),axis=1)
re_map = pd.DataFrame.from_records(re_map)
gatis_roads[list(re_map.columns)] = re_map.fillna(gatis_roads[list(re_map.columns)])

# convert int
def convert_int(x):
    try:
        return int(x)
    except:
        return None
gatis_roads['thru_lanes'] = streets['exist_lanes'].apply(lambda x: convert_int(x))

In [12]:
# replace all NaNs with None
gatis_roads = gatis_roads.where(gatis_roads.notnull(), None)
all_gatis_edges.append(gatis_roads)

## Crosswalks



| Name | Note |
| - | - |
| school | drop |
| created_by | drop |
| intersection_id | drop |
| created_date | drop |
| subtype | keep |
| marking_size | drop |
| description | drop |
| segment_id | drop, not unique |
| alternate_id | drop |
| school_desc | drop |
| direction | drop |
| asset_condition | drop |
| color | drop |
| objectid | drop |
| globalid | drop |
| short_line_type | **keep to filter for just crosswalks** |
| location_status | **add to status** |
| other_area | drop |
| last_work_date | **add to check_date** |
| shape_length | drop |
| modified_by | drop |
| marking_text_note | drop |
| crew_assigned | drop |
| material | drop |
| school_name | drop |
| markings_short_line_id | **add to reference_ids** |
| cbd | drop  |
| hin | drop |
| signal_intersection | **add to vehicle_traffic_control** |
| modified_date | drop |
| school_zone_id | drop |
| number_of_assets | drop |
| comments | drop |
| geometry | **kept as is** |

In [13]:
crosswalks = gpd.read_file("data/austin_all_data.gpkg", layer='TRANSPORTATION.markings_short_line_20250825', mask=bbox).explode()

keep_cols = [
    'subtype','short_line_type','location_status',
    'last_work_date','markings_short_line_id','signal_intersection',
    'geometry'
    ]
crosswalks = crosswalks[keep_cols]

# only keep crosswalks
crosswalks = crosswalks[crosswalks['short_line_type'] == 'CROSSWALK']

# only keep active
crosswalks = crosswalks[crosswalks['location_status'] == 'ACTIVE']

crosswalks.reset_index(drop=True,inplace=True)

In [14]:
# create empty gatis geodataframes with the same structure
gatis_crosswalks = geojson_to_geopandas(gatis_edges_geojson,'edge','crossing')
gatis_crosswalks = create_empty_gdf_like(gatis_crosswalks,crosswalks)

In [15]:
# OR put in a status column with open, under construction
reference_id = crosswalks[['markings_short_line_id']].to_dict(orient='records')
gatis_crosswalks['reference_ids'] = [[{'source': 'austin', 'source_url': '',**item}] for item in reference_id]

# converting this to the GATIS compliant format
gatis_crosswalks['check_date'] = crosswalks['last_work_date'].dt.strftime(date_format="%Y-%m-%d")

# these match up to GATIS
gatis_crosswalks['visual_markings'] = crosswalks['subtype'].str.lower()

gatis_crosswalks.loc[crosswalks['signal_intersection']=='Y',['vehicle_traffic_control','cross_vehicle_traffic_control','ped_traffic_control']] = 'standard_signal'

In [16]:
# replace all NaNs with None
gatis_crosswalks = gatis_crosswalks.where(gatis_crosswalks.notnull(), None)
all_gatis_edges.append(gatis_crosswalks)

## Multi-Use Paths and Trails

| Name | Note |
|---|---|
| urban_trail_system_name | **add to facility_name** |
| created_by | drop |
| urban_trail_name | drop |
| urban_trail_network_id | **add to reference_ids** |
| location | drop |
| phase_simple | **add to status** |
| created_date | drop, looks different than year_open |
| construction_manager | drop |
| year_open | **add to build_date** |
| length_miles | drop |
| objectid | drop |
| county | drop |
| build_status | drop, use phase_simple |
| shape_length | drop |
| modified_by | drop |
| urban_trail_feature | **keep, use to separate out features and populate the bridge field** |
| project_sponsor | drop |
| managing_agency_name | drop |
| priority_2023utp | drop |
| trail_surface_type | **add to surface** |
| agency_type | drop |
| width | **add to width** |
| modified_date | drop |
| city_municipal | drop |
| urban_trail_type | **kee to drop shared laens and separated bike lanes (duplicative)** |
| geometry | **kept as is** |

In [17]:
mups = gpd.read_file("data/austin_all_data.gpkg", layer='TRANSPORTATION_urban_trails_network_20250825', mask=bbox).explode()
keep_cols = [
    "urban_trail_system_name", "urban_trail_network_id", "phase_simple",
    "year_open", "urban_trail_feature", "trail_surface_type", "width",
    "urban_trail_type", "geometry"
]
mups = mups[keep_cols]

# drop potenial mups
mups = mups[mups['phase_simple'] != 'Potential'].copy()

# separate steps
steps = mups[mups['urban_trail_feature']=='Stairs']
mups = mups[mups['urban_trail_feature']!='Stairs']

# NOTE there are crossings present in mups that should be added to gatis_crossings
additional_crossings = mups[mups['urban_trail_feature']=='Crossing']
mups = mups[mups['urban_trail_feature']!='Crossing']

# drop shared lanes / protected bike lane because these are likely already in the streets file
# NOTE: need confirmation on this
mups = mups[mups['urban_trail_type'].isin(['On-Street – Protected Bike Lane', 'On-Street - Shared Lane'])==False]


mups.reset_index(drop=True,inplace=True)
additional_crossings.reset_index(drop=True,inplace=True)
steps.reset_index(drop=True,inplace=True)

In [18]:
# create empty gatis geodataframes with the same structure
gatis_mups = geojson_to_geopandas(gatis_edges_geojson,'edge','multi_use_path')
gatis_mups = create_empty_gdf_like(gatis_mups,mups)

gatis_additional_crossings = geojson_to_geopandas(gatis_edges_geojson,'edge','crossing')
gatis_additional_crossings = create_empty_gdf_like(gatis_additional_crossings,additional_crossings)

gatis_steps = geojson_to_geopandas(gatis_edges_geojson, "edge", "steps")
gatis_steps = create_empty_gdf_like(gatis_steps,steps)

In [19]:
# add surface_material (needed minimal processing)
remap_surfaces = {
    "stalock": "other",
    "boards-woods": "other",
    "crushed_stone": "gravel"
}
gatis_mups['surface_material'] = mups["trail_surface_type"].str.lower().map(remap_surfaces)
gatis_additional_crossings['surface_material'] = additional_crossings["trail_surface_type"].str.lower().map(remap_surfaces)
gatis_steps['surface_material'] = steps["trail_surface_type"].str.lower().map(remap_surfaces)

# add facility name
gatis_mups['facility_name'] = mups["urban_trail_system_name"]
gatis_additional_crossings['facility_name'] = additional_crossings["urban_trail_system_name"]
gatis_steps['facility_name'] = steps["urban_trail_system_name"]

# add reference_ids
reference_id = mups[['urban_trail_network_id']].to_dict(orient='records')
gatis_mups['reference_ids'] = [[{'source': 'austin',**item}] for item in reference_id]
reference_id = additional_crossings[['urban_trail_network_id']].to_dict(orient='records')
gatis_additional_crossings['reference_ids'] = [[{'source': 'austin',**item}] for item in reference_id]
reference_id = steps[['urban_trail_network_id']].to_dict(orient='records')
gatis_steps['reference_ids'] = [[{'source': 'austin',**item}] for item in reference_id]

# add status
def process_phase_simple(x):
    if x in ["Existing","Active"]:
        return "yes"
    elif x in ["Construction"]:
        return "under construction"
    else:
        return "unknown"
gatis_mups['status'] = mups["phase_simple"].apply(lambda x: process_phase_simple(x))
gatis_additional_crossings['status'] = additional_crossings["phase_simple"].apply(lambda x: process_phase_simple(x))
gatis_steps['status'] = steps["phase_simple"].apply(lambda x: process_phase_simple(x))

# date_built
gatis_mups['date_built'] = mups["year_open"]
gatis_additional_crossings['date_built'] = additional_crossings["year_open"]
gatis_steps['date_built'] = steps["year_open"]

# add bridge
gatis_mups.loc[mups['urban_trail_feature']=='Bridge','bridge'] = mups['urban_trail_feature']
gatis_additional_crossings.loc[additional_crossings['urban_trail_feature']=='Bridge','bridge'] = additional_crossings["urban_trail_feature"]
gatis_steps.loc[steps['urban_trail_feature']=='Bridge','bridge'] = steps["urban_trail_feature"]

# add width (and convert to inches)
gatis_mups['width'] = pd.to_numeric(mups['width']) * 12
gatis_additional_crossings['width'] = pd.to_numeric(additional_crossings['width']) * 12
gatis_steps['width'] = pd.to_numeric(steps['width']) * 12

In [20]:
gatis_mups = gatis_mups.where(gatis_mups.notnull(), None)
all_gatis_edges.append(gatis_mups)
gatis_additional_crossings = gatis_additional_crossings.where(gatis_additional_crossings.notnull(), None)
all_gatis_edges.append(gatis_additional_crossings)
gatis_steps = gatis_steps.where(gatis_steps.notnull(), None)
all_gatis_edges.append(gatis_steps)

## Sidewalks

| Name | Keep |
| - | - |
| rating_curbramp_id | drop |
| project_manager | drop |
| created_by | drop |
| public_works_id | drop |
| functional_improve_cut | drop |
| work_assignment | drop |
| zipcode | drop |
| sub_project_id | drop |
| sidewalks_id | keep |
| functional_improve_lift | drop |
| year_built | keep |
| width_sidewalk | **add to width** |
| created_date | drop |
| construction_constraint | drop |
| construction_manager | drop |
| mxaddresscode | drop |
| managing_agency | drop |
| urban_trail_network | drop |
| leveraging | drop |
| assessment_type | drop |
| assigned_for_construction_date | drop |
| mobility_annual_plan | drop |
| pedestrian_facility_type | **keep to remove shared use paths (duplicative)** |
| rating_overall | drop |
| sidewalk_surface | **add to surface** |
| objectid | drop |
| globalid | drop |
| construction_start | drop |
| rating_curbramp | drop |
| full_street_name | drop |
| shape_length | drop |
| modified_by | drop |
| rating_no_veg | drop |
| assessment_date | **add to check_date** |
| status | drop |
| date_construction_completed | **add to build_date** |
| planning_status_rehab | drop |
| address_description | drop |
| service_plan_note | drop |
| ada_year_route_checked | drop, almost sounds like an ADA assessment date? |
| planning_status | drop |
| notes | drop |
| created_user | drop |
| last_edited_date | drop |
| back_of_curb | drop |
| rating_final | drop |
| stamped_finish | drop |
| width | drop |
| modified_date | drop |
| core_sw_network | drop |
| last_edited_user | drop |
| fund_department_unit | drop |
| colored_finish | drop |
| functional_improve_grind | drop |
| mapsco | drop |
| contract | drop |
| sidewalk_segments_length | keep |
| council_district | drop |
| functional_condition | drop |
| mxlocation | drop |
| functional_improve_rehab | drop |
| geometry | **kept as is** |

In [21]:
sidewalks = gpd.read_file("data/austin_all_data.gpkg", layer='Sidewalks_20250825', mask=bbox).explode()
keep_cols = [
    "sidewalks_id", "year_built", "width_sidewalk",
    "pedestrian_facility_type", "sidewalk_surface", "assessment_date",
    "date_construction_completed", "geometry"
]

# only keep existing_sidewalk and driveway
# NOTE some of the different types like SHARED_STREET look like pedestrian lanes
sidewalks = sidewalks[sidewalks['pedestrian_facility_type'].isin(['EXISTING_SIDEWALK','DRIVEWAY'])]

sidewalks.reset_index(drop=True,inplace=True)

In [22]:
# create empty gatis geodataframes with the same structure
gatis_sidewalks = geojson_to_geopandas(gatis_edges_geojson,'edge','sidewalk')
gatis_sidewalks = create_empty_gdf_like(gatis_sidewalks,sidewalks)

In [23]:
# add reference_ids
reference_id = sidewalks[['sidewalks_id']].to_dict(orient='records')
gatis_sidewalks['reference_ids'] = [[{'source': 'austin',
**item}] for item in reference_id]

# add street_name
gatis_sidewalks['street_name'] = sidewalks['full_street_name']

# add width
gatis_sidewalks['width'] = (pd.to_numeric(sidewalks['width']) * 12)

# add surface_material
convert_sidewalk_surface = {
    'CONCRETE': 'concrete',
    'EXPOSED_AGGREGATE': 'concrete',
    'COLORED': 'other',
    'ASPHALT': 'asphalt',
    'PAVER-BRICK': 'paving_stones',
    'PAVER-GRANITE':'paving_stones',
    'EXPERIMENTAL': 'other',
    'PAVER-SANDSTONE': 'paving_stones',
    'CRUSHED_STONE': 'gravel',
    'PAVER-CONCRETE': 'paving_stones',
    'STAMPED': 'other',
    'PLASTIC_PANEL': 'other',
    'PERVIOUS_CONCRETE': 'concrete',
    'RUBBERIZED': 'other'
}
gatis_sidewalks['surface_material'] = sidewalks['sidewalk_surface'].map(convert_sidewalk_surface)

# get length
gatis_sidewalks['measured_length'] = sidewalks['sidewalk_segments_length']

# get year built
gatis_sidewalks.loc[sidewalks['year_built']!='0','date_built'] = sidewalks['year_built']

# get check_date
gatis_sidewalks['check_date'] = sidewalks['assessment_date'].dt.strftime(date_format="%Y-%m-%d")

In [24]:
gatis_sidewalks = gatis_sidewalks.where(gatis_sidewalks.notnull(), None)
all_gatis_edges.append(gatis_sidewalks)

## Combine GeoDataFrames

In [25]:
gatis_edges_gdf = pd.concat(all_gatis_edges,axis=0)
gatis_nodes_gdf = pd.concat(all_gatis_nodes,axis=0)

# reset index
gatis_edges_gdf.reset_index(drop=True,inplace=True)
gatis_nodes_gdf.reset_index(drop=True,inplace=True)

# NOTE these IDs are for demonstration purposes
# give ids
gatis_edges_gdf['edge_id'] = list(range(0,gatis_edges_gdf.shape[0]))
gatis_nodes_gdf['node_id'] = list(range(0,gatis_nodes_gdf.shape[0]))

C:\Users\tanner.passmore.ctr\AppData\Local\Temp\ipykernel_35108\1336736901.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gatis_edges_gdf = pd.concat(all_gatis_edges,axis=0)


## Topology Correction
Ideally, GIS data will already be topologicaly correct, if it's not then sometimes topology correction can be used. The following cells join sidewalks, curb ramps, and crosswalks where it makes sense. Some additional processing would be needed to join the topologically correct the other features.

### Join curb ramps nodes to all crossings endpoints within 0.0001 decimal degress with a virtual link

In [26]:
crossings = gatis_edges_gdf[gatis_edges_gdf['edge_type']=='crossing']

In [27]:
# extract curb ramp nodes
curb_ramps = gatis_nodes_gdf[gatis_nodes_gdf['node_type']=='curb_ramp']

# extract crossing endpoints
def extract_endpoint(geom,idx):
    if geom.geom_type != 'LineString':
        raise Exception("Invalid geometry:",geom.geom_type)
    return geom.coords[idx]
crossing_endpoints_1 = gpd.GeoDataFrame({'edge_id':crossings['edge_id'],'geometry':crossings['geometry'].apply(lambda geom: Point(extract_endpoint(geom,0)))},crs='epsg:4326')
crossing_endpoints_1['idx'] = 0
crossing_endpoints_2 = gpd.GeoDataFrame({'edge_id':crossings['edge_id'],'geometry':crossings['geometry'].apply(lambda geom: Point(extract_endpoint(geom,-1)))},crs='epsg:4326')
crossing_endpoints_2['idx'] = -1
crossing_endpoints = pd.concat([crossing_endpoints_1,crossing_endpoints_2])
crossing_endpoints.reset_index(drop=True,inplace=True)

# buffer nodes by tolerance
crossing_endpoints['buffer_geometry'] = crossing_endpoints['geometry'].buffer(0.0001)#.map(lambda x: Point(x).buffer(0.0001))
crossing_endpoints.set_geometry('buffer_geometry',inplace=True,crs='epsg:4326')

# find candidate matches with an intersect
overlay = gpd.overlay(curb_ramps[['node_id','geometry']],crossing_endpoints,how='intersection').drop(columns=['geometry'])

# bring in both geoms
overlay = overlay.merge(curb_ramps[['node_id','geometry']],on='node_id').merge(crossing_endpoints,on=['edge_id','idx'],suffixes=("_node","_edge"))

# calulate distance (FYI not using great circle distance but the tolerances are small)
overlay['distance'] = gpd.GeoSeries(overlay['geometry_edge'],crs='epsg:4326').distance(gpd.GeoSeries(overlay['geometry_node'],crs='epsg:4326'))

# get minimum for each
minimum = overlay.loc[overlay.groupby(['node_id'])['distance'].idxmin()]

# create virtual edges
minimum = minimum.apply(lambda x: LineString([x['geometry_node'].coords[0][0:2],x['geometry_edge'].coords[0][0:2]]), axis=1)
minimum = gpd.GeoDataFrame({'geometry':minimum},crs='epsg:4326')

C:\Users\tanner.passmore.ctr\AppData\Local\Temp\ipykernel_35108\852272495.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  crossing_endpoints['buffer_geometry'] = crossing_endpoints['geometry'].buffer(0.0001)#.map(lambda x: Point(x).buffer(0.0001))
C:\Users\tanner.passmore.ctr\AppData\Local\Temp\ipykernel_35108\852272495.py:27: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  overlay['distance'] = gpd.GeoSeries(overlay['geometry_edge'],crs='epsg:4326').distance(gpd.GeoSeries(overlay['geometry_node'],crs='epsg:4326'))


In [28]:
gatis_virtual_links = geojson_to_geopandas(gatis_edges_geojson,'edge','virtual_link')
gatis_virtual_links = create_empty_gdf_like(gatis_virtual_links,minimum)

# re-index
gatis_virtual_links.index = range(gatis_edges_gdf.shape[0],gatis_edges_gdf.shape[0]+gatis_virtual_links.shape[0])
gatis_edges_gdf = pd.concat([gatis_edges_gdf,gatis_virtual_links])

### Join curb ramps nodes to the nearest sidewalk (interpolate on sidewalk edges and add a virtual link)

In [29]:
sidewalks = gatis_edges_gdf[gatis_edges_gdf['edge_type']=='sidewalk']
curb_ramps = gatis_nodes_gdf[gatis_nodes_gdf['node_type']=='curb_ramp']

# buffer curb_ramp nodes by tolerance (0.0001 degrees)
curb_ramps['buffer_geometry'] = curb_ramps.buffer(0.0001)
curb_ramps.set_geometry('buffer_geometry',crs='epsg:4326',inplace=True)

# intersect to get sidewalk candidates
candidate_matches = gpd.overlay(sidewalks[['edge_id','geometry']],curb_ramps[['node_id','buffer_geometry']],how='intersection').drop(columns=['geometry'])
candidate_matches = candidate_matches.merge(curb_ramps[['node_id','geometry']],on='node_id').merge(sidewalks[['edge_id','geometry']],on='edge_id',suffixes=("_node","_edge"))
candidate_matches

# interpolate a point on sidewalk
candidate_matches['projection'] = candidate_matches.apply(lambda x: x['geometry_edge'].project(x['geometry_node']), axis=1)
candidate_matches['interpolated_point'] = candidate_matches.apply(lambda x: shapely.line_interpolate_point(x['geometry_edge'],x['projection']), axis=1)

# calculate distances
candidate_matches['match_dist'] = candidate_matches.apply(lambda x: x['geometry_node'].distance(x['interpolated_point']), axis=1)

# accept nearest sidewalk point for each curb ramp node
minimum = candidate_matches.loc[candidate_matches.groupby('node_id')['match_dist'].idxmin()]

# create virtual links
minimum = minimum.apply(lambda x: LineString([x['geometry_node'].coords[0][0:2],x['interpolated_point'].coords[0][0:2]]), axis=1)
minimum = gpd.GeoDataFrame({'geometry':minimum},crs='epsg:4326')

# add to gatis edges
gatis_virtual_links = geojson_to_geopandas(gatis_edges_geojson,'edge','virtual_link')
gatis_virtual_links = create_empty_gdf_like(gatis_virtual_links,minimum)

# re-index
gatis_virtual_links.index = range(gatis_edges_gdf.shape[0],gatis_edges_gdf.shape[0]+gatis_virtual_links.shape[0])
gatis_edges_gdf = pd.concat([gatis_edges_gdf,gatis_virtual_links])

C:\Users\tanner.passmore.ctr\AppData\Local\Temp\ipykernel_35108\3737498778.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  curb_ramps['buffer_geometry'] = curb_ramps.buffer(0.0001)


## Export Data

In [30]:
# export a gpkg version for viewing in qgis
gatis_edges_gdf.loc[:,gatis_edges_gdf.isna().all(axis=0)==False].to_file("austin_sample.gpkg",layer="gatis_edges")
gatis_nodes_gdf.loc[:,gatis_nodes_gdf.isna().all(axis=0)==False].to_file("austin_sample.gpkg",layer="gatis_nodes")

# export the GeoJSON version
with open("austin_sample_edges.geojson","w") as f:
    f.write(gatis_edges_gdf.to_json(na="drop",indent=2,drop_id=True,to_wgs84=True))
with open("austin_sample_nodes.geojson","w") as f:
    f.write(gatis_nodes_gdf.to_json(na="drop",indent=2,drop_id=True,to_wgs84=True))

In [31]:
# # exports a leaflet map that you can view in your browser, don't do this for the full dataset because it won't finish running
# # requires a custom module called gatis_sample_data that isn't currently public
# from gatis_sample_data import utils
# bboxes = []
# bboxes.append(shapely.from_wkt("POLYGON((-97.748244 30.278771, -97.736805 30.278771, -97.736805 30.267221, -97.748244 30.267221, -97.748244 30.278771))"))
# bboxes.append(shapely.from_wkt("POLYGON((-97.729976 30.258163, -97.718538 30.258163, -97.718538 30.246611, -97.729976 30.246611, -97.729976 30.258163))"))
# for idx, bbox in enumerate(bboxes):
#     if bbox is not None:
#         masked_edges = gatis_edges_gdf[gatis_edges_gdf['geometry'].intersects(bbox)]
#         masked_nodes = gatis_nodes_gdf[gatis_nodes_gdf['geometry'].intersects(bbox)]
#         m = utils.display_layers(masked_edges, masked_nodes, edge_categories="edge_type", node_categories="node_type")
#         m.save(f"scratch_map_{idx}.html")